In [74]:
import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly
from ipywidgets import widgets 
from IPython.display import display

from plotly.widgets import GraphWidget
plotly.tools.set_credentials_file(username='amcdonne', api_key='TgHHDa2sn3SAj65ZH0dz')

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

<IPython.core.display.Javascript object>

In [2]:
D_2017_1_01 = 'https://raw.githubusercontent.com/Amcdonne95/SeniorDesignData/master/Drive%20Auto%20Receivables%20Trust/DRIVE%202017-1/2017/05%20May/2017.06.15_Drive%20Auto%20Receivables%20Trust%202017-1%20Data%20Tape.csv'

In [93]:
df = pd.read_csv(D_2017_1_01, memory_map=True, low_memory=False)

In [59]:
def mapData(deal, mapDict, col_list):
    for i in col_list:
        deal[str(i)] = pd.to_numeric(deal[str(i)], errors='coerce')
        deal[str(i)] = deal[str(i)].map(mapDict[str(i)])
    return deal

In [68]:
cols = ['vehicleTypeCode', 'vehicleNewUsedCode']
df = mapData(df, groupDict, cols)

In [61]:
df.head()

,assetTypeNumber,assetNumber,reportingPeriodBeginningDate,reportingPeriodEndingDate,originatorName,originationDate,originalLoanAmount,originalLoanTerm,loanMaturityDate,originalInterestRatePercentage,interestCalculationTypeCode,originalInterestRateTypeCode,originalInterestOnlyTermNumber,originalFirstPaymentDate,underwritingIndicator,gracePeriodNumber,paymentTypeCode,subvented,vehicleManufacturerName,vehicleModelName,vehicleNewUsedCode,vehicleModelYear,vehicleTypeCode,vehicleValueAmount,vehicleValueSourceCode,obligorCreditScoreType,obligorCreditScore,obligorIncomeVerificationLevelCode,obligorEmploymentVerificationCode,coObligorIndicator,paymentToIncomePercentage,obligorGeographicLocation,assetAddedIndicator,remainingTermToMaturityNumber,reportingPeriodModificationIndicator,servicingAdvanceMethodCode,reportingPeriodBeginningLoanBalanceAmount,nextReportingPeriodPaymentAmountDue,reportingPeriodInterestRatePercentage,nextInterestRatePercentage,servicingFeePercentage,servicingFlatFeeAmount,otherServicerFeeRetainedByServicer,otherAssessedUncollectedServicerFeeAmount,scheduledInterestAmount,scheduledPrincipalAmount,otherPrincipalAdjustmentAmount,reportingPeriodActualEndBalanceAmount,reportingPeriodScheduledPaymentAmount,totalActualAmountPaid,actualInterestCollectedAmount,actualPrincipalCollectedAmount,actualOtherCollectedAmount,servicerAdvancedAmount,interestPaidThroughDate,zeroBalanceEffectiveDate,zeroBalanceCode,currentDelinquencyStatus,primaryLoanServicerName,mostRecentServicingTransferReceivedDate,assetSubjectDemandIndicator,assetSubjectDemandStatusCode,repurchaseAmount,DemandResolutionDate,repurchaserName,repurchaseReplacementReasonCode,chargedoffPrincipalAmount,recoveredAmount,modificationTypeCode,paymentExtendedNumber,repossessedIndicator,repossessedProceedsAmount
0,SC,"=""6345520""",05-01-2017,05-31-2017,SC,10/2013,28203.21,72,11/2019,0.1932,1,1,-,12/2013,True,2,2,1,DODGE TRUCK,Journey,New,2013,SUV,23109.0,98,Bureau,592,2,2,False,0.139531,MI,-,31,False,1,17115.35,1586.34,0.1932,0.1932,0.0,-,-,0.00,256.2709,408.2091,0.0,16672.09,664.48,669.75,226.49,443.26,0.00,-,05-04-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
1,SC,"=""6345701""",05-01-2017,05-31-2017,SC,11/2013,13786.00,72,05/2020,0.2499,1,1,-,12/2013,True,1,2,0,FORD,Focus,Used,2009,Car,11998.0,98,Bureau,570,2,2,True,0.138530,SC,-,37,False,1,12184.17,499.93,0.2499,0.2499,0.0,-,-,365.42,173.2527,198.0173,0.0,12151.05,371.27,380.00,342.02,33.12,4.86,-,05-17-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
2,SC,"=""6345822""",05-01-2017,05-31-2017,SC,11/2013,12515.00,60,08/2019,0.2100,1,1,-,12/2013,True,1,2,0,GMC LIGHT DUTY,Yukon,Used,2003,SUV,11998.0,98,Bureau,574,2,2,True,0.077623,CO,-,28,False,1,10288.10,583.43,0.2100,0.2100,0.0,-,-,105.00,92.9365,245.6335,0.0,10124.13,338.57,341.54,177.57,163.97,0.00,-,05-15-2017,-,-,0,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
3,SC,"=""6346124""",05-01-2017,05-31-2017,SC,10/2013,16699.00,60,11/2018,0.2100,1,1,-,12/2013,True,2,2,0,TOYOTA,Tundra,Used,2007,Truck,15225.0,98,Bureau,629,3,2,True,0.114215,OK,-,19,False,1,7828.79,990.21,0.2100,0.2100,0.0,-,-,22.50,124.0593,327.7007,0.0,7619.25,451.76,434.75,225.21,209.54,0.00,-,05-14-2017,-,-,8,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,False,-
4,SC,"=""1006458""",05-01-2017,05-31-2017,SC,05/2012,21404.16,72,08/2018,0.2499,1,1,-,06/2012,True,1,2,0,JEEP,Grand Cherokee,Used,2006,SUV,17998.0,98,Bureau,-,3,3,False,0.154751,NV,-,16,False,1,10385.66,1277.14,0.2499,0.2499,0.0,-,-,429.72,130.9025,445.5375,0.0,10385.66,576.44,0.00,0.00,0.00,0.00,-,04-30-2017,-,-,15,SC,-,False,-,-,-,-,-,0.0,0.0,-,0,True,0.00


In [94]:
obligorMetrics = ['obligorCreditScore', 'obligorIncomeVerificationLevelCode', 'obligorEmploymentVerificationCode','paymentToIncomePercentage']
creditMetrics = ['originalLoanTerm', 'remainingTermToMaturityNumber','originalInterestRatePercentage','LTV','subvented']
vehicleMetrics = ['vehicleNewUsedCode', 'vehicleManufacturerName','vehicleModelName', 'vehicleModelYear','vehicleTypeCode',
                  'obligorGeographicLocation']
groupDict = {
    'vehicleNewUsedCode':{
        1:'New',
        2:'Used'
    },
    'vehicleTypeCode':{
        1:'Car',
        2:'Truck',
        3:'SUV',
        4:'Motorcycle',
        98:'Other',
        99:'Unknown'
    },
    'subvented':{
        0:'Not Subvented',
        1:'Rate Subvention',
        2:'Cash Subvention',
        3:'Rate & Cash Subvention',
        98:'Other Subvention'
    },
    'obligorIncomeVerificationLevelCode':{
    1:'Not Stated, Not Verified',
    2:'Stated, Not Verified',
    3:'Stated, Verified',
    4:'Stated, Verified: 12 M',
    5:'Stated, Verified: 24 M'
    },
    'obligorEmploymentVerificationCode':{
    1:'Not Stated, Not Verified',
    2:'Stated, Not Verified',
    3:'Stated, Third Party Verified'
    }
    #'vehicleValueAmount': 
}

In [95]:
def creditStats(metrics, data, mappingDict):
    WA = lambda x: np.average(x, axis=0, weights = data.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount']/(data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount']/data['vehicleValueAmount']) #Loan to value ratio
    creditLis = []
    dfCredit = pd.DataFrame()
    for i in range(0, len(metrics)):
        #data[metrics[i]] = pd.to_numeric(data[metrics[i]], errors='coerce')
        if metrics[i] == 'subvented':
            data[metrics[i]] = data[metrics[i]].fillna(3)
            data[['pctDeal']].groupby(data[metrics[i]]).sum().rename(index=groupDict[metrics[i]]).head(3)
            stats = data[['pctDeal']].groupby(data[metrics[i]]).sum().rename(index=groupDict[metrics[i]]).head(3)
            #del stats.index.name
            print(stats)
            dfCredit = dfCredit.append(stats)
            continue
        creditLis.append([WA(data[metrics[i]])])
    print(creditLis)
    return creditLis #original term, remaining term, original interenst rate, LTV

In [69]:
def vehicleStats(metrics, data, mappingDict):
    WA = lambda x: np.average(x, axis=0, weights = data.loc[x.index, 'reportingPeriodActualEndBalanceAmount'])
    data['pctDeal'] = (data['reportingPeriodActualEndBalanceAmount']/(data['reportingPeriodActualEndBalanceAmount'].sum()))
    data['LTV'] = (data['originalLoanAmount']/data['vehicleValueAmount']) #Loan to value ratio
    dfVehicleStats = pd.DataFrame()
    for i in range(0, len(metrics)):
        if metrics[i] in ['vehicleModelYear', 'vehicleValueAmount']:
            data[metrics[i]] = pd.to_numeric(data[metrics[i]], errors='coerce')
        if metrics[i] in['vehicleManufacturerName', 'vehicleModelName', 'obligorGeographicLocation', 'vehicleModelYear', 'vehicleTypeCode']:
            if metrics[i] == 'vehicleTypeCode':
                data[['pctDeal']].groupby(data[metrics[i]]).sum().head(3)#.rename(index=groupDict[metrics[i]])
            (data[['pctDeal']].groupby(data[metrics[i]]).sum()).sort_values(['pctDeal'], ascending=False).head(3)
            dfNew = (data[['pctDeal']].groupby(data[metrics[i]]).sum()).sort_values(['pctDeal'], ascending=False).head(3)
            #del dfNew.index.name
            dfVehicleStats = dfVehicleStats.append(dfNew)
            #print(dfNew)
            '''
            if metrics[i] == 'vehicleModelYear':
                yearRange = [str(data[metrics[i]].min()) + '-' + str(data[metrics[i]].max())]
                pd.DataFrame(yearRange, index=['Model Year Range'])
                dfYearRange = pd.DataFrame(yearRange, index=['Model Year Range'])
                print(dfYearRange)
            continue
            '''
        else:
            data[['pctDeal']].groupby(data[metrics[i]]).sum().head(3)#.rename(index=groupDict[metrics[i]])
            stats = data[['pctDeal']].groupby(data[metrics[i]]).sum()#.rename(index=groupDict[metrics[i]])
            #del stats.index.name
            dfVehicleStats = dfVehicleStats.append(stats)
            #print(stats)
    return dfVehicleStats.reset_index()

In [96]:
#vehicleMetrics = vehicleStats(vehicleMetrics, df, groupDict)
creditMetrics = creditStats(creditMetrics, df, groupDict)

            pctDeal
subvented          
0          0.974983
1          0.024131
1;  2      0.000849
[[69.950342053988834], [68.33007931510393], [0.19283320594904907], [1.0859827828660842]]


In [97]:
creditMetrics.head()

AttributeError: 'list' object has no attribute 'head'